In [2]:
import socket

In [3]:
s1 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s2 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s3 = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

In [4]:
s1.connect(('192.168.1.10', 7))

In [5]:
s2.connect(('192.168.1.10', 7))

In [6]:
s3.connect(('192.168.1.10', 7))

In [13]:
# Send data to server
data = bytearray([1,2,3,4]);

In [20]:
s1.send(data);

In [18]:
s2.send(data);

In [19]:
s3.send(data);